<a href="https://colab.research.google.com/github/Angel-dash/Abstractive-Text-Summarization/blob/main/DL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [2]:
!unzip /content/drive/MyDrive/MiniProjectdata/data.zip


Archive:  /content/drive/MyDrive/MiniProjectdata/data.zip
  inflating: cnn_dailymail/test.csv  
  inflating: cnn_dailymail/train.csv  
  inflating: cnn_dailymail/validation.csv  


In [29]:
import numpy as np
import pandas as pd
import re
from bs4 import BeautifulSoup
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from nltk.corpus import stopwords
from tensorflow.keras.layers import Input, LSTM, Embedding, Dense, Concatenate, TimeDistributed, Bidirectional
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping
import warnings
pd.set_option("display.max_colwidth", 200)
warnings.filterwarnings("ignore")
import textwrap
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [4]:
data_train=pd.read_csv('/content/cnn_dailymail/train.csv')
data_test=pd.read_csv('/content/cnn_dailymail/test.csv')
data_validation=pd.read_csv('/content/cnn_dailymail/validation.csv')

Data preprocessing

In [5]:
data_train.head(3)

,id,article,highlights
0,0001d1afc246a7964130f43ae940af6bc6c57f01,"By . Associated Press . PUBLISHED: . 14:11 EST, 25 October 2013 . | . UPDATED: . 15:36 EST, 25 October 2013 . The bishop of the Fargo Catholic Diocese in North Dakota has exposed potentially hundr...","Bishop John Folda, of North Dakota, is taking time off after being diagnosed .\nHe contracted the infection through contaminated food in Italy .\nChurch members in Fargo, Grand Forks and Jamestown..."
1,0002095e55fcbd3a2f366d9bf92a95433dc305ef,"(CNN) -- Ralph Mata was an internal affairs lieutenant for the Miami-Dade Police Department, working in the division that investigates allegations of wrongdoing by cops. Outside the office, author...","Criminal complaint: Cop used his role to help cocaine traffickers .\nRalph Mata, an internal affairs lieutenant, allegedly helped group get guns .\nHe also arranged to pay two assassins in a murde..."
2,00027e965c8264c35cc1bc55556db388da82b07f,"A drunk driver who killed a young woman in a head-on crash while checking his mobile phone has been jailed for six years. Craig Eccleston-Todd, 27, was driving home from a night at a pub when he r...","Craig Eccleston-Todd, 27, had drunk at least three pints before driving car .\nWas using phone when he veered across road in Yarmouth, Isle of Wight .\nCrashed head-on into 28-year-old Rachel Titl..."


In [6]:
data = pd.concat([data_train, data_test, data_validation ])
train_df = data

In [7]:
train_df.shape

(311971, 3)

In [8]:
sample=data_train.sample()
print("****Article****")
print(sample.article.values)
print('****Highlights****')
print(sample.highlights.values)

****Article****
["By . Keith Gladdis . PUBLISHED: . 06:19 EST, 27 September 2012 . | . UPDATED: . 17:07 EST, 27 September 2012 . Trial: Justin Lee Collins (pictured today) arrives for the second day of his trial where he was accused of trying to push his ex-girlfriend in front of a taxi . TV star Justin Lee Collins tried to . push his girlfriend in front of a taxi on a night out with his friend, . comedian Alan Carr, a court heard yesterday. Collins, 38, is accused of subjecting . Anna Larke to a campaign of verbal and physical abuse during a . nine-month relationship last year. It is claimed he forced Miss Larke to . sleep facing him and ordered her to throw away DVDs featuring actors she . found attractive and compile a dossier of her former lovers. A jury has been told Miss Larke, 38, . is a recovering alcoholic with a history of depression. Sonia Woodley, . QC, defending at St Alban’s Crown Court, said Miss Larke had told . Collins about three suicide attempts including an occasion

Checking for Null values


In [9]:
data_train.isna().sum()

id            0
article       0
highlights    0
dtype: int64

Checking for duplicates


In [10]:
data_train.duplicated(subset=['article','highlights']).sum()

3098

In [11]:
data_train=data_train.drop_duplicates(subset=['article','highlights'])
data_train.shape

(284015, 3)

In [12]:
data_train.duplicated(subset=['article','highlights']).sum()

0

In [43]:
data['article'][:10]

0    by . associated press . published: . 14:11 est, 25 october 2013 . | . updated: . 15:36 est, 25 october 2013 . the bishop of the fargo catholic diocese in north dakota has exposed potentially hundr...
1    (cnn) -- ralph mata was an internal affairs lieutenant for the miami-dade police department, working in the division that investigates allegations of wrongdoing by cops. outside the office, author...
2    a drunk driver who killed a young woman in a head-on crash while checking his mobile phone has been jailed for six years. craig eccleston-todd, 27, was driving home from a night at a pub when he r...
3    (cnn) -- with a breezy sweep of his pen president vladimir putin wrote a new chapter into crimea's turbulent history, committing the region to a future returned to russian domain. sixty years prio...
4    fleetwood are the only team still to have a 100% record in sky bet league one as a 2-0 win over scunthorpe sent graham alexander’s men top of the table. the cod army are playi

In [41]:
data['article'] = data['article'].str.lower()
data['highlights'] = data['highlights'].str.lower()

print(data)

                                             id  \
0      0001d1afc246a7964130f43ae940af6bc6c57f01   
1      0002095e55fcbd3a2f366d9bf92a95433dc305ef   
2      00027e965c8264c35cc1bc55556db388da82b07f   
3      0002c17436637c4fe1837c935c04de47adb18e9a   
4      0003ad6ef0c37534f80b55b4235108024b407f0b   
...                                         ...   
13363  e93f721ba4949f21f33549c4a21d55ff456af979   
13364  8df19a570ad14119a7d00f3bbe864fedf8c1691d   
13365  2fdd5f89aa26e91ceea9b0ef264abfcfc3e6fa2e   
13366  530d7b18d7a715b368b0745f9dfebfe353adeda8   
13367  42d6f6e10a45ff469fbe3ef7345838aa45dcdd61   

                                                                                                                                                                                                       article  \
0      by . associated press . published: . 14:11 est, 25 october 2013 . | . updated: . 15:36 est, 25 october 2013 . the bishop of the fargo catholic diocese in north dakota has